In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import * 

# Without Bias

In [2]:
X_train = [[0,0,1,1] for x in range(20)]+[[0,1,1,0] for x in range(20)]
X_test = [[0,0,1,1] for x in range(20)]+[[0,1,1,0] for x in range(20)]
y_train = [0 for x in range(20)]+[1 for x in range(20)]
y_test =[0 for x in range(20)]+[1 for x in range(20)]

### Without Bias

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

In [ ]:
with open('results/version6/error_by_epoch_unbiased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [3]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})


erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.5107421875
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.505859375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.505859375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4912109375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4912109375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4912109375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4912109375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4609375
erro phase encoding 0.7109375

erro HSGS 1.0
erro encoding weight 0.966796875
erro encoding input 0.4609375
erro phase encoding 0.7109375

erro HSG

In [4]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [5]:
with open('results/version7/weights_unbiased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [6]:
with open('results/version7/weights_unbiased_v7.json') as json_file:
    weights = json.load(json_file)

In [7]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=False, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.475
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.5
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.425
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.625
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.475
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.5
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.55
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.6
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.475
erro phase encoding 0.0
ERROS HSGS             0.5
ERROS ENCODING WEIGHT  0.0
ERROS ENCODING INPUT   0.5075
ERROS PHASE ENCODING   0.0


In [8]:
with open('results/version7/experiments_unbiased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

## BIAS

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})
                                    

In [ ]:
with open('results/version6/error_by_epoch_biased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [9]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})


erro HSGS 0.552734375
erro encoding weight 0.751953125
erro encoding input 0.73828125
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.744140625
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.736328125
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
erro phase encoding 0.5048828125

erro HSGS 0.52734375
erro encoding weight 0.71875
erro encoding input 0.734375
e

In [10]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [11]:
with open('results/version7/weights_biased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

In [12]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=True, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

erro HSGS 0.625
erro encoding weight 0.0
erro encoding input 0.575
erro phase encoding 0.0
erro HSGS 0.4
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
erro HSGS 0.65
erro encoding weight 0.0
erro encoding input 0.6
erro phase encoding 0.0
erro HSGS 0.375
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
erro HSGS 0.475
erro encoding weight 0.0
erro encoding input 0.475
erro phase encoding 0.0
erro HSGS 0.475
erro encoding weight 0.0
erro encoding input 0.5
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.4
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
erro HSGS 0.5
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
erro HSGS 0.45
erro encoding weight 0.0
erro encoding input 0.45
erro phase encoding 0.0
ERROS HSGS             0.495
ERROS ENCODING WEIGHT  0.0
ERROS ENCODING INPUT   0.4800000000000001
ERROS PHASE E

In [13]:
with open('results/version7/experiments_biased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)